# Bidimensional boundary element method

This examples uses BEM_base to solve the Helmholtz and Laplace equations in bidimensional space using polynomial shape functions and NURBS curves to describe the boundary and interpolate domain variables (velocity potential and its flux for the Helmholtz equation and temperature and its flux for the Laplace equation). The first example which will be solved is acoustic duct problem, which has an analytical solution. If both ends of the acoustic duct are closed, then a Neumann boundary condition ($q = \bar{q}$) must be applied to every element. Let the boundary condition of the wall in one of the extremities not be null and that extremity will act as a piston. The analytical resonance frequencies for such a duct are given by
\begin{equation}
    F_{closed\ duct}\ (n,L) = \frac{\pi\ n\ c}{L}
\end{equation}
where $c$ is the wave propagation speed, $n=0,1,2,...$ is the mode number and $L$ is the length of the duct. The frequency is in rad/s, divide it to $2\ pi$ to obtain the frequency in Hz. The duct which will be studied has length = 100 mm and diameter = 10 mm. The speed of sound is $c = 343$ m/s and only the first non-zero mode will be studied ($n=1$).


In [1]:
workspace() #start with a clean workspace
c = 343*1000 # Speed of propagation in mm/s
F_closed(n,L,c) = pi*n*c/L # Analytical resonance frequency in rad/s
for n=1:3
    println("Mode n = $(n), L = $(L=100) m, F = $(F_closed(n,L,c)/(2*pi)) Hz.")
end

Mode n = 1, L = 100 m, F = 1715.0 Hz.
Mode n = 2, L = 100 m, F = 3430.0 Hz.
Mode n = 3, L = 100 m, F = 5145.0 Hz.


To solve this problem using constant bidimensional elements, the geometry must be described to BEM_base. Initially, the module for the constant bidimensional element will be loaded.

In [20]:
# First, include the module const2D
include("../src/const2D/const2D.jl")
using const2D
using KrylovMethods

## Now, to define a new geometry, first one must declare the points,
# segments, boundary conditions, etc...
n = 1; # Mode number
L = 100; # Length of the duct in mm
d = 10; # Diameter of the duct in mm
c = 343*1000; # Speed of wave propagation in mm/s
k = F_closed(n,L,c)/c; # Resonance wave number
points = [1 0 0; 2 L 0; 3 L d; 4 0 d];
segments = [1 1 2 0; 2 2 3 0; 3 3 4 0; 4 4 1 0];
ne = 4; # Number of element per segment
MESH = [1 ne; 2 ne; 3 ne; 4 ne];
BCFace = [1 1 0; 2 1 0; 3 1 0; 4 1 1]; # Face 4 will act like a piston
fc = [0];
# Apply the discretization technique and builds the problems matrices
#for the geometrical points, physical nodes, elements' connectivity and
#boundary conditions
info = const2D.format_dad(points,segments,MESH,BCFace);
# Now, the domain points will be created
n_pint = 100
PONTOS_int = zeros(n_pint,3)
for i = 1:n_pint
	PONTOS_int[i,:] = [i ((100-0.1)/n_pint)*i d/2]
end
u,q,uint,qint = const2D.solve(info,PONTOS_int,fc,BCFace,k);
uH,qH,uintH,qintH = const2D.solveH(info,PONTOS_int,fc,BCFace,k);


Building A and b matrices using the traditional colocation BEM for constant elements.


  0.151297 seconds (92.97 k allocations: 3.193 MiB)
Evaluating values at domain points.
  0.137268 seconds (250.38 k allocations: 5.369 MiB)


LoadError: [91merror compiling solveH: unsupported or misplaced expression "import" in function solveH[39m

In [8]:
# Comparing the results
using PyPlot

phi_closed(x,n,L,c) = cos.(n*pi*(x./L))
figure()
plot(PONTOS_int[:,2],phi_closed(PONTOS_int[:,2],n,L,c),label="Mode n=$(n), F = $(F_closed(n,L,c)/(2*pi)) Hz")
hold(1)
plot(PONTOS_int[:,2],real(uint)./maximum(abs.(uint[:,1])),label="BEM")
#plot(PONTOS_int[:,2],real(uint)./maximum(abs.(uint[:,1])),label="H-BEM")

legend()
xlabel("distance [m]")
ylabel("normalized velocity potential")
grid(1)
hold(0)